## Grid navigation problem

#### simulating a product picking operation in a warehouse and finding the shortest route between all products

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

In [ ]:
import pandas as pd
df=pd.read_csv("/content/drive/MyDrive/Inventory Managment project/AZ.csv")
df.head()

,SKU,Product Name,Category,Quantity,Aisle,Shelf,Bin,Reorder Level
0,SKU1000,Portable Charger,Electronics,77,A2,7,7,19
1,SKU1001,Mouse Pad,Accessories,187,A2,7,3,14
2,SKU1002,USB Cable,Electronics,49,A3,3,6,14
3,SKU1003,Pen Set,Stationery,6,A3,8,8,5
4,SKU1004,Office Chair,Furniture,132,A2,1,10,8


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   SKU            500 non-null    object
 1   Product Name   500 non-null    object
 2   Category       500 non-null    object
 3   Quantity       500 non-null    int64 
 4   Aisle          500 non-null    object
 5   Shelf          500 non-null    int64 
 6   Bin            500 non-null    int64 
 7   Reorder Level  500 non-null    int64 
dtypes: int64(4), object(4)
memory usage: 31.4+ KB


In [ ]:
l=dict()
for i in df.columns[1:]:
    if i!="Quantity":
        l[i]=list(df[i].unique())
print(l)

{'Product Name': ['Portable Charger', 'Mouse Pad', 'USB Cable', 'Pen Set', 'Office Chair', 'Drawer Set', 'Highlighter Pack', 'Microwave Oven', 'Stapler', 'Toaster', 'Backpack', 'Bookshelf', 'Water Bottle', 'Wireless Mouse', 'Sticky Notes', 'Electric Kettle', 'Coffee Maker', 'Standing Desk', 'Filing Cabinet', 'Blender', 'Travel Mug', 'Notebook', 'Noise-Canceling Headset', 'Laptop Sleeve', 'Laptop Stand'], 'Category': ['Electronics', 'Accessories', 'Stationery', 'Furniture', 'Appliances'], 'Aisle': ['A2', 'A3', 'A1', 'A4', 'A5'], 'Shelf': [7, 3, 8, 1, 2, 9, 5, 10, 4, 6], 'Bin': [7, 3, 6, 8, 10, 9, 1, 5, 4, 2], 'Reorder Level': [19, 14, 5, 8, 16, 18, 9, 20, 13, 17, 10, 7, 11, 12, 6, 15]}


In [ ]:
for i in df["Aisle"].unique():
    print(df[df["Aisle"]==i]["Category"].unique())

['Electronics' 'Accessories' 'Furniture' 'Appliances' 'Stationery']
['Electronics' 'Stationery' 'Accessories' 'Furniture' 'Appliances']
['Furniture' 'Stationery' 'Appliances' 'Electronics' 'Accessories']
['Stationery' 'Furniture' 'Electronics' 'Appliances' 'Accessories']
['Furniture' 'Appliances' 'Stationery' 'Accessories' 'Electronics']


#### Distribution of products in each Aisle by category

In [ ]:
px.bar(df,x='Aisle',y='Category',color='Category')

In [ ]:
def plot_cat_val(col:str,m:str="for each",plot_type="bar",sort=False):
    filter=df[col].value_counts().sort_values(ascending=sort).reset_index(name="count")
    if plot_type=="pie":
        return px.pie(filter,values='count',names=col,color=col,title=f'Number of products {m} {col}')
    elif plot_type=="bar":
        return px.bar(filter,x=col,y="count",color=col,title=f'Number of products {m} {col}')

In [ ]:
plot_cat_val("Category","for each","bar")

In [ ]:
plot_cat_val('Category',"for each","pie")

In [ ]:
plot_cat_val("Aisle","for each","bar")

In [ ]:
plot_cat_val("Aisle","for each","pie")

#### Quantity of products in each Aisle and Category

In [ ]:
px.sunburst(df, path=[ 'Aisle','Category'], values='Quantity',
                  color='Aisle')

### Reorganizing products by category

In [ ]:
#creating a dictionary containing categories and aisles
categories=list(df["Category"].unique())
aisles=list(df["Aisle"].unique())
cat_aisle=dict(list(zip(categories,aisles)))
cat_aisle

{'Electronics': 'A2',
 'Accessories': 'A3',
 'Stationery': 'A1',
 'Furniture': 'A4',
 'Appliances': 'A5'}

In [ ]:
for i in cat_aisle.keys():
    print(i,cat_aisle[i])
    mask=df["Category"]==i

    df.loc[mask,"Aisle"]=cat_aisle[i]


Electronics A2
Accessories A3
Stationery A1
Furniture A4
Appliances A5


In [ ]:
px.sunburst(df, path=[ 'Aisle','Category'], values='Quantity',
                  color='Aisle')

### Basic dashboard for monitoring products

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

#### Displaying product quantity by Aisle and Product Name

In [ ]:
def plot_val(b):
    with output:
        output.clear_output(wait=True)  # Clear previous plot
        selected_product = product.value  # Get selected product
        if selected_product:  # Only plot if a product is selected
            # Filter data for the selected product
            data_to_plot = df[df["Product Name"] == selected_product]
            q=data_to_plot["Quantity"].sum()
            # Create the bar plot using matplotlib
            plt.figure(figsize=(8, 4))
            plt.bar(data_to_plot["Aisle"], data_to_plot["Quantity"], color='skyblue')
            plt.title(f"Quantity of {selected_product} by Aisle : {q}")
            plt.xlabel("Aisle")
            plt.ylabel("Quantity")
            plt.show()

In [ ]:
# widget to select the product
select_aisle = widgets.Dropdown(
    options=list(df["Aisle"].sort_values().unique()),
    description='Aisle:',
    disabled=False
)
button = widgets.Button(
    description='display quantity',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

In [ ]:
output=widgets.Output()
def update_output(change):
    """
      a function to update the output when the values of Aisle and products are changed
    """
    # Effacer l'affichage précédent
    output.clear_output()
    with output:
      pr=list(df[df["Aisle"]==select_aisle.value]["Product Name"].unique())
      global product
      product=widgets.Dropdown(
      placeholder='Choose product',
      options=pr,
      description='product:',
      disabled=False
      )
      display(product)
      display(button)


In [ ]:
select_aisle.observe(update_output, names='value')
# Mettre à jour l'affichage initial
update_output(None)
# Affichage du layout
display(select_aisle,output)
button.on_click(plot_val)

Dropdown(description='Aisle:', options=('A1', 'A2', 'A3', 'A4', 'A5'), value='A1')

Output()

#### simulating warehouse

In [ ]:
# Example data for columns and rows (representing aisles and shelves)
col = sorted(list(df['Aisle'].unique()))  # Unique aisles from the data
row = sorted(list(df['Shelf'].unique()))  # Unique shelves from the data
n = len(col) - 1
m = len(row) - 1

In [ ]:
import plotly.graph_objects as go



def afficher_entrepot(n, m,res=None, entry=(0, 0), exit=None, shelves=None, obstacles=None):
    """
    afficher_entrepot(n, m, entry, exit, shelves, obstacles) : function to display a warehouse layout on a grid
      Arguments:
        n : int, grid width based on Aisle values
        m : int, grid height based on Shelf values
        entry : tuple, coordinates of the entry/loading point
        exit : tuple, coordinates of the exit/shipping point (default to bottom-right corner)
        shelves : list of tuples, coordinates representing storage shelf locations
        obstacles : list of tuples, coordinates representing obstacles
    """
    # Set the default exit point if not specified
    if exit is None:
        exit = (n, m)

    # Initialize the figure
    fig = go.Figure()

    # Draw the grid lines
    for i in range(n + 1):
        fig.add_shape(
            type="line",
            x0=i, y0=0, x1=i, y1=m,
            line=dict(color="black", width=1)
        )
    for j in range(m + 1):
        fig.add_shape(
            type="line",
            x0=0, y0=j, x1=n, y1=j,
            line=dict(color="black", width=1)
        )
    # Initialize starting coordinates
    if res!=None:
      x, y = [entry[0]], [entry[1]]

    # Update coordinates based on movement commands
      for i in res:
        if i == 'R':  # Move right
            x.append(x[-1] + 1)
            y.append(y[-1])
        elif i == 'U':  # Move up
            x.append(x[-1])
            y.append(y[-1] + 1)

    # Plot the path
      fig.add_trace(go.Scatter(
          x=x, y=y,
          mode="lines+markers",
          line=dict(color="yellow", width=2),
          marker=dict(color="yellow", size=8),
          name="Path"
      ))

    # Mark entry point
    fig.add_trace(go.Scatter(
        x=[entry[0]], y=[entry[1]],
        mode="text+markers",
        text=["Entry"],
        marker=dict(color="green", size=10),
        textposition="bottom center",
        textfont=dict(size=12, color="green")
    ))

    # Mark exit point
    fig.add_trace(go.Scatter(
        x=[exit[0]], y=[exit[1]],
        mode="text+markers",
        text=["Exit"],
        marker=dict(color="orange", size=10),
        textposition="top center",
        textfont=dict(size=12, color="orange")
    ))

    # Add storage shelves if specified
    if shelves is not None:
        x_shelves, y_shelves = zip(*shelves)
        fig.add_trace(go.Scatter(
            x=x_shelves, y=y_shelves,
            mode="markers",
            marker=dict(color="blue", size=10, symbol="square"),
            name="Shelves"
        ))

    # Add obstacles if specified
    if obstacles is not None:
        x_obstacles, y_obstacles = zip(*obstacles)
        fig.add_trace(go.Scatter(
            x=x_obstacles, y=y_obstacles,
            mode="markers",
            marker=dict(color="black", size=10, symbol="x"),
            name="Obstacles"
        ))

    # Set custom axis labels for Aisle (x-axis) and Shelf (y-axis)
    fig.update_xaxes(
        tickvals=list(range(len(col))),
        ticktext=col,
        range=[-0.5, n + 0.5],
        showticklabels=True,
        title="Aisle"
    )
    fig.update_yaxes(
        tickvals=list(range(len(row))),
        ticktext=row,
        range=[-0.5, m + 0.5],
        showticklabels=True,
        title="Shelf",
        scaleanchor="x"  # Keep square cells
    )

    # Set layout properties
    fig.update_layout(
        title="Warehouse Layout",
        showlegend=True,
        width=800, height=800,
        margin=dict(l=20, r=20, t=40, b=20)
    )

    # Show the plot
    fig.show()


#### generating all shelves coordinates

In [ ]:
s=[]
for j in range(len(col)):
    for i in range(len(row)+1):
        t=(j,i)
        s.append(t)

#### plotting the warehouse map

In [ ]:
afficher_entrepot(n, m,entry=(0, 0) ,shelves=s)

#### storing the products in a list

#### searching by SKU Stock keeping unit id

In [ ]:
def add(b):
  l.append(skus.value)
  with output2:
    #output2.clear_output(wait=True)
    print(l)
def clr(b):
  l.clear()
  with output2:
    output2.clear_output(wait=True)
    print(l)

In [ ]:
unique_sku=list(df['SKU'].unique())
output2=widgets.Output()
skus=widgets.Combobox(
      placeholder='Choose product',
      options=unique_sku,
      description='product:',
      disabled=False
      )
button2= widgets.Button(
    description='add product',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
button3= widgets.Button(
    description='clear list',
    disabled=False,
)
global l
l=[]
def find_product(b):
    selected_sku = skus.value
    if selected_sku:
        with output2:
            output2.clear_output(wait=True)
            display(df[df["SKU"]==selected_sku])


# Function to clear the list and output
def clr(b):
    global l
    l.clear()
    with output2:
        output2.clear_output()
        print("SKU list cleared")
display(skus,output2)
display(button2)
button2.on_click(find_product)
#display(button3)
#button2.on_click(clr)

Combobox(value='', description='product:', options=('SKU1000', 'SKU1001', 'SKU1002', 'SKU1003', 'SKU1004', 'SK…

Output()

Button(description='add product', icon='check', style=ButtonStyle(), tooltip='Click me')

#### finding the shortest path to all the products

In [ ]:
def find_path():
  pass